<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Supp_Fig_7/virus_abundances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q anndata
import anndata
import numpy as np
import random
import pandas as pd
import json
import os
import glob
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

def flatten(xss):
    """
    Function to flatten array of arrays.
    """
    return [x for xs in xss for x in xs]

# Load data
The count matrices were generated [here](https://github.com/pachterlab/LSCHWCP_2023/tree/main/Notebooks/align_macaque_PBMC_data/7_virus_host_captured_dlist_cdna_dna) (masked) and [here](https://github.com/pachterlab/LSCHWCP_2023/tree/main/Notebooks/align_macaque_PBMC_data/1_virus_no_mask) (unmasked).

In [ ]:
# Download count matrices from Caltech data

In [ ]:
# Load count matrix generated with masked host genome and transcriptome
palmdb_adata = anndata.read("virus_host-captured_dlist_cdna_dna.h5ad")
palmdb_adata

In [ ]:
# Load count matrix generated without masking host sequences
palmdb_adata_raw = anndata.read("virus_no_mask.h5ad")
palmdb_adata_raw

#### Load ID to taxonomy mapping

In [ ]:
# Load virus ID to taxonomy mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
phylogeny_data = pd.read_csv("ID_to_taxonomy_mapping.csv")

# Drop columns not needed here
phylogeny_data = phylogeny_data.drop("ID", axis=1).drop("strandedness", axis=1).drop_duplicates()
phylogeny_data

___

# Plot fractions of positive time points and animals for each virus

In [ ]:
# Macaque only and shared viruses in the same order as shown in Fig. 6d
virs = [
    # Macaque only
    'u39566', 'u102540', 'u11150', 'u10',
    # Shared
    'u288819',
    'u290519',
    'u10240',
    'u183255',
    'u1001',
    'u100291',
    'u103829',
    'u110641',
    'u181379',
    'u202260',
    'u135858',
    'u101227',
    'u100188',
    'u27694',
    'u34159',
    'u100245',
    'u10015',
    'u100733',
    'u100173',
    'u100196',
    'u100599',
    'u100644',
    'u100296',
    'u100017',
    'u100002',
    'u100012',
    'u100024',
    'u100048',
    'u100302',
    'u100074',
    'u100289',
    'u100026',
    'u100111',
    'u100139',
    'u100154',
    'u100251',
    'u100177',
    'u100215',
    'u100049',
    'u100000',
    'u100001',
    'u100007',
    'u100004',
    'u100011',
    'u100093',
    'u100116',
    'u100019',
    'u100076',
    'u100028',
    'u100153',
    'u100031',
    'u100145',
    'u102324',
    'u134800'
]

In [ ]:
# Minimum fraction of positive cells to count virus as present in sample
threshold = 0.0005

In [ ]:
animals = palmdb_adata.obs["donor_animal"].unique()
animal_fracs = []
for vir in virs:
    pos_fracs = []
    for animal_id in animals:
        # Get number of positive cells per animal
        num_pos_cells = palmdb_adata[palmdb_adata.obs["donor_animal"] == animal_id, palmdb_adata.var.index.values == vir].X.sum()
        total_num_cells = len(palmdb_adata.obs[palmdb_adata.obs["donor_animal"] == animal_id])
        pos_fracs.append(num_pos_cells/total_num_cells)

    # Get number of animals with > threshold positive cells
    pos_animals = sum(i >= threshold for i in pos_fracs)

    animal_fracs.append(np.array(pos_animals) / len(animals))

In [ ]:
tps = palmdb_adata.obs["dpi_clean_merged"].unique()
tp_fracs = []
for vir in virs:
    pos_fracs = []
    for tp in tps:
        # Get number of positive cells per time point
        num_pos_cells = palmdb_adata[palmdb_adata.obs["dpi_clean_merged"] == tp, palmdb_adata.var.index.values == vir].X.sum()
        total_num_cells = len(palmdb_adata.obs[palmdb_adata.obs["dpi_clean_merged"] == tp])
        pos_fracs.append(num_pos_cells/total_num_cells)

    # Get number of animals with > threshold positive cells
    pos_tps = sum(i >= threshold for i in pos_fracs)

    tp_fracs.append(np.array(pos_tps) / len(tps))

In [ ]:
fig, axs = plt.subplots(figsize=(20,9), nrows=2, sharex=True)
fontsize = 16

ax = axs[0]
ax.bar(virs, animal_fracs, color="lightgrey", ec="black")
ax.set_ylabel("Fraction of postive animals", fontsize=fontsize)
ax.set_xticklabels("")
ax.tick_params(axis="both", labelsize=fontsize)
ax.axhline(0.5, c="black", ls="--", lw=1)
ax.margins(x=0.005)

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)
ax.xaxis.grid(False)

ax = axs[1]
ax.bar(virs, tp_fracs, color="lightgrey", ec="black")
ax.set_ylabel("Fraction of postive time points", fontsize=fontsize)
ax.set_xlabel("Virus ID", fontsize=fontsize)
ax.tick_params(axis="both", labelsize=fontsize)
ax.set_xticklabels(virs, rotation=45, ha="right", fontsize=fontsize-2)
ax.axhline(0.5, c="black", ls="--", lw=1)
ax.margins(x=0.005)

plt.subplots_adjust(hspace=0.05)

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)
ax.xaxis.grid(False)

plt.savefig("virus_frequencies.png", dpi=300, bbox_inches="tight")

fig.show()

___
# Create Venn diagram to show overlap between viruses

In [ ]:
# https://github.com/tctianchi/pyvenn
!wget https://raw.githubusercontent.com/tctianchi/pyvenn/master/venn.py
import venn

In [ ]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def get_venn_counts(df):
    result_dict = {}
    # Find all possible combinations of viruses, starting with the one combining all viruses
    for ps_tuple in list(powerset(df))[1:][::-1]:
        ps = list(ps_tuple)
        try:
            # Count instances
            result_dict['_'.join(ps)] = int(df[ps].min(axis=1).sum())

            # Remove counted instances from df
            bool_list = list(map(bool, 1-df[ps].min(axis=1).values)) # Invert array and convert to boolean
            df = df[bool_list]

        # If combination not available because all rows have been counted
        except KeyError:
            result_dict['_'.join(ps)] = 0

    return result_dict

In [ ]:
df = pd.DataFrame()

df["u10"] = nd(palmdb_adata[:, palmdb_adata.var.index == "u10"].X.todense())
df["u102540"] = nd(palmdb_adata[:, palmdb_adata.var.index == "u102540"].X.todense())
df["u11150"] = nd(palmdb_adata[:, palmdb_adata.var.index == "u11150"].X.todense())
df["u202260"] = nd(palmdb_adata[:, palmdb_adata.var.index == "u202260"].X.todense())

In [ ]:
labels_temp = get_venn_counts(df)
labels_temp

In [ ]:
names = ['u10', 'u102540', 'u11150', 'u202260']

In [ ]:
def rename_labels(labels_temp):
    labels = {}
    for key in labels_temp:
        if key == "u10_u102540_u11150_u202260":
            labels["1111"] = labels_temp[key]
        if key == "u102540_u11150_u202260":
            labels["0111"] = labels_temp[key]
        if key == "u10_u11150_u202260":
            labels["1011"] = labels_temp[key]
        if key == "u10_u102540_u202260":
            labels["1101"] = labels_temp[key]
        if key == "u10_u102540_u11150":
            labels["1110"] = labels_temp[key]
        if key == "u11150_u202260":
            labels["0011"] = labels_temp[key]
        if key == "u102540_u202260":
            labels["0101"] = labels_temp[key]
        if key == "u102540_u11150":
            labels["0110"] = labels_temp[key]
        if key == "u10_u202260":
            labels["1001"] = labels_temp[key]
        if key == "u10_u11150":
            labels["1010"] = labels_temp[key]
        if key == "u10_u102540":
            labels["1100"] = labels_temp[key]
        if key == "u202260":
            labels["0001"] = labels_temp[key]
        if key == "u11150":
            labels["0010"] = labels_temp[key]
        if key == "u102540":
            labels["0100"] = labels_temp[key]
        if key == "u10":
            labels["1000"] = labels_temp[key]

    return labels

In [ ]:
len(labels_temp)

In [ ]:
# Rename labels keys to match venn annotation
labels = rename_labels(labels_temp)
len(labels)

In [ ]:
labels

In [ ]:
fig, ax = venn.venn4(labels, names=names)
plt.savefig('vir_pos_cell_overlap.png', dpi=300, bbox_inches= 'tight')
fig.show()

In [ ]:
palmdb_adata[:, palmdb_adata.var.index == "u202260"].X.sum()

# Same for count matrix wihtout host masking (used for regression models)

In [ ]:
df_raw = pd.DataFrame()

df_raw["u10"] = nd(palmdb_adata_raw[:, palmdb_adata_raw.var.index == "u10"].X.todense())
df_raw["u102540"] = nd(palmdb_adata_raw[:, palmdb_adata_raw.var.index == "u102540"].X.todense())
df_raw["u11150"] = nd(palmdb_adata_raw[:, palmdb_adata_raw.var.index == "u11150"].X.todense())
df_raw["u202260"] = nd(palmdb_adata_raw[:, palmdb_adata_raw.var.index == "u202260"].X.todense())

In [ ]:
labels_raw_temp = get_venn_counts(df_raw)
labels_raw_temp

In [ ]:
labels_raw = rename_labels(labels_raw_temp)
len(labels_raw)

In [ ]:
fig, ax = venn.venn4(labels_raw, names=names)
plt.savefig('vir_pos_cell_overlap_unmasked.png', dpi=300, bbox_inches= 'tight')
fig.show()